In [100]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
from pathlib import Path
from PIL import Image

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [101]:
# https://stackoverflow.com/questions/8598673/how-to-save-a-pylab-figure-into-in-memory-file-which-can-be-read-into-pil-image
def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf, dpi = 64) # Requerido para que la imagen sea 512x512
    buf.seek(0)
    img = Image.open(buf)
    return img

In [147]:
def transformGridImage(sample):
    brain_vol_data = sample.get_fdata()
    fig_rows = 4
    fig_cols = 4
    n_subplots = fig_rows * fig_cols
    n_slice = brain_vol_data.shape[2]
    step_size = n_slice // n_subplots
    plot_range = n_subplots * step_size
    start_stop = int((n_slice - plot_range) / 2)

    fig, axs = plt.subplots(fig_rows, fig_cols, figsize=[10, 10], facecolor='black', dpi=64)
    fig.set_size_inches(8, 8)
    fig.set_dpi(64)
    
    for idx, img in enumerate(range(start_stop, plot_range, step_size)):
        axs.flat[idx].imshow(ndi.rotate(brain_vol_data[:, :, img], 90), cmap='gray')
        axs.flat[idx].axis('off')
        
    plt.tight_layout()

    image = fig2img(fig)
    # plt.savefig('filename.png', dpi=64) # guarda en 512x512
    # plt.show()

    # image.show()

    plt.close(fig) # Para que no muestre la imágen
    
    # return sample
    return image

In [148]:
# CreateGrid transform
class CreateGrid(object):
    """Creates a grid from the image
    """
    def __init__(self):
        True

    def __call__(self, sample):
        return transformGridImage(sample)

In [158]:
class ToLabelOutput(object):
    def __call__(self, label):
        if label == "CN":
            return torch.tensor([1, 0, 0])
        elif label == "AD":
            return torch.tensor([0, 1, 0])
        else:
            return torch.tensor([0, 0, 1])

In [161]:
class ADNIDataset(Dataset):
    """ADNI dataset."""

    def __init__(self, csv_file, root_dir, transform=None, target_transform = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        studyID = self.landmarks_frame.iloc[idx, 0]
        subjectID = self.landmarks_frame.iloc[idx, 1]
        processFormat = self.landmarks_frame.iloc[idx, 7]
        date = self.landmarks_frame.iloc[idx, 9]
        diagnosis = self.landmarks_frame.iloc[idx, 2]
        
        filename = None
        
        rglob = str(studyID)+'/'+'*.nii'
        print(rglob)
        samples = 0
        for path in Path('ADNI-Full-PostProc').rglob(rglob):
            filename = str(path)
            samples =+ 1
            
        if samples > 1:
            raise "Mas de un sample. Error"

        if not filename:
            raise "Not found: " + filename
            
        brain_vol = nib.load(filename)
    
        # sample = {'image': brain_vol, 'diagnosis': diagnosis}

        image = brain_vol
        label = diagnosis
        
        if self.transform:
            image = self.transform(image)
            
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [162]:
adniDataset = ADNIDataset('ADNI-Full-PostProc/ADNI-FULL-PostProc_11_29_2022_UniformResolution.csv', '', transform = CreateGrid())
len(adniDataset)

2219

In [163]:
image, label = adniDataset[518]
# brain_vol_data = sample['image'].get_fdata()
# diagnosis = sample['diagnosis']
image.show()
label

I59400/*.nii


'AD'

# Con dos transformaciones

In [164]:
adniDataset = ADNIDataset('ADNI-Full-PostProc/ADNI-FULL-PostProc_11_29_2022_UniformResolution.csv', '', transform = transforms.Compose([CreateGrid(), transforms.ToTensor()]), target_transform =ToLabelOutput() )
len(adniDataset)

2219

In [165]:
image, label = adniDataset[518]

I59400/*.nii


In [166]:
print(image)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 

In [167]:
print(label)

tensor([0, 1, 0])
